# P4Lang Tutorials of FABRIC

This notebook walks the user through setting up a FABRIC eperiment that is suitiable for completing the P4 tutorials created by [P4Lang](https://github.com/p4lang/tutorials). The tutorials were origianlly designed to use a mininet topology. This example replaces the mininet topology with a FABRIC experiemnt topology that may span multiple sites across the FABRIC testbed.

Additional resources:
- [FABRIC Knowledge Base](https://learn.fabric-testbed.net/)
- [FABRIC Forums](https://learn.fabric-testbed.net/forums/)
- [P4Lang Tutorials](https://github.com/p4lang/tutorials)
- [P4Lang YouTube Presentations](https://www.youtube.com/channel/UCOQAFkDKucJWr-KafdJsdIQ)

## Basic FABRIC Slice Configuration

In [1]:
import json
import traceback
from fabrictestbed_extensions.fablib.fablib import fablib

In [2]:
try:
    print(f"{fablib.list_sites()}")
except Exception as e:
    print(f"Exception: {e}")

Name      CPUs  Cores    RAM (G)    Disk (G)       Basic (100 Gbps NIC)    ConnectX-6 (100 Gbps x2 NIC)    ConnectX-5 (25 Gbps x2 NIC)    P4510 (NVMe 1TB)    Tesla T4 (GPU)    RTX6000 (GPU)
------  ------  -------  ---------  -------------  ----------------------  ------------------------------  -----------------------------  ------------------  ----------------  ---------------
MASS         6  132/192  1180/1536  60170/60600    355/381                 2/2                             2/2                            10/10               2/2               3/3
SALT         6  180/192  1492/1536  60440/60600    367/381                 2/2                             1/2                            10/10               2/2               3/3
NCSA         6  188/192  1520/1536  60580/60600    378/381                 2/2                             2/2                            10/10               2/2               3/3
UTAH        10  298/320  2472/2560  116200/116400  618/635                 2/2

## Configure Slice Parameters

This section builds the experiment slice 

<img src="figs/fabric_slice.png" width="800"/>



In [3]:
from ipaddress import ip_address, IPv4Address, IPv6Address, IPv4Network, IPv6Network

# Slice 
slice_name = 'P4Lang_Tutorial'

[site1,site2,site3] = ['UTAH', 'UTAH', 'UTAH']
# fablib.get_random_sites(count=3)
print(f"Sites: {site1},{site2},{site3}")

# Switches
s1_name = "s1"
s2_name = "s2"
s3_name = "s3"

switch_cores = 2
switch_ram = 8
switch_disk = 100

# Hosts
h1_name = "h1"
h2_name = "h2"
h3_name = "h3"
h11_name = "h11"
h22_name = "h22"

h1_subnet=IPv4Network('10.0.1.0/24')
h1_addr=IPv4Address('10.0.1.1')

h11_subnet=IPv4Network('10.0.1.0/24')
h11_addr=IPv4Address('10.0.1.11')

h2_subnet=IPv4Network('10.0.2.0/24')
h2_addr=IPv4Address('10.0.2.2')

h22_subnet=IPv4Network('10.0.2.0/24')
h22_addr=IPv4Address('10.0.2.22')

h3_subnet=IPv4Network('10.0.3.0/24')
h3_addr=IPv4Address('10.0.3.3')

host_cores = 2
host_ram = 8
host_disk = 10

net_h1_name = 'net_h1'
net_h2_name = 'net_h2'
net_h3_name = 'net_h3'

net_h11_name = 'net_h11'
net_h22_name = 'net_h22'

net_s1_s2_name = 'net_s1_s2'
net_s2_s3_name = 'net_s2_s3'
net_s1_s3_name = 'net_s1_s3'

# All node properties
image = 'default_ubuntu_20'

Sites: UTAH,UTAH,UTAH


### Create the Slice

In [4]:
try:
    #Create Slice
    slice = fablib.new_slice(name=slice_name)
    
    # Add switch node s1
    s1 = slice.add_node(name=s1_name, site=site1,  image=image, 
                        cores=switch_cores, ram=switch_ram, disk=switch_disk)
    s1.set_capacities(cores=switch_cores, ram=switch_ram, disk=switch_disk)
    s1_iface_local = s1.add_component(model='NIC_Basic', name="s1_local_nic").get_interfaces()[0]
    s11_iface_local = s1.add_component(model='NIC_Basic', name="s11_local_nic").get_interfaces()[0]
    s1_iface_to_s2 = s1.add_component(model='NIC_Basic', name="s1_switch_nic1").get_interfaces()[0]
    s1_iface_to_s3 = s1.add_component(model='NIC_Basic', name="s1_switch_nic2").get_interfaces()[0]

    # Add switch node s2
    s2 = slice.add_node(name=s2_name, site=site2,  image=image, 
                        cores=switch_cores, ram=switch_ram, disk=switch_disk)
    s2_iface_local = s2.add_component(model='NIC_Basic', name="s2_local_nic").get_interfaces()[0]
    s22_iface_local = s2.add_component(model='NIC_Basic', name="s22_local_nic").get_interfaces()[0]
    s2_iface_to_s1 = s2.add_component(model='NIC_Basic', name="s2_switch_nic1").get_interfaces()[0]
    s2_iface_to_s3 = s2.add_component(model='NIC_Basic', name="s2_switch_nic2").get_interfaces()[0]
    
    # Add switch node s3
    s3 = slice.add_node(name=s3_name, site=site3,  image=image, 
                        cores=switch_cores, ram=switch_ram, disk=switch_disk)
    s3_iface_local = s3.add_component(model='NIC_Basic', name="s3_local_nic").get_interfaces()[0]
    s3_iface_to_s1 = s3.add_component(model='NIC_Basic', name="s3_switch_nic1").get_interfaces()[0]
    s3_iface_to_s2 = s3.add_component(model='NIC_Basic', name="s3_switch_nic2").get_interfaces()[0]    
    
    # Add host node h1
    h1 = slice.add_node(name=h1_name, site=site1, image=image,
                        cores=host_cores, ram=host_ram, disk=host_disk)
    h1_iface = h1.add_component(model='NIC_Basic', name="h1_nic").get_interfaces()[0]
    
    # Add host node h11
    h11 = slice.add_node(name=h11_name, site=site1, image=image,
                        cores=host_cores, ram=host_ram, disk=host_disk)
    h11_iface = h11.add_component(model='NIC_Basic', name="h11_nic").get_interfaces()[0]
    
    # Add host node h2
    h2 = slice.add_node(name=h2_name, site=site2, image=image,
                        cores=host_cores, ram=host_ram, disk=host_disk)
    h2_iface = h2.add_component(model='NIC_Basic', name="h2_nic").get_interfaces()[0]
    
    # Add host node h22
    h22 = slice.add_node(name=h22_name, site=site2, image=image,
                        cores=host_cores, ram=host_ram, disk=host_disk)
    h22_iface = h22.add_component(model='NIC_Basic', name="h22_nic").get_interfaces()[0]
    
    # Add host node h3
    h3 = slice.add_node(name=h3_name, site=site3, image=image,
                        cores=host_cores, ram=host_ram, disk=host_disk)
    h3_iface = h3.add_component(model='NIC_Basic', name="h3_nic").get_interfaces()[0]
    
    #Add swtich networks
    switch_net1 = slice.add_l2network(name=net_s1_s2_name, interfaces=[s1_iface_to_s2, s2_iface_to_s1])
    swtich_net2 = slice.add_l2network(name=net_s2_s3_name, interfaces=[s2_iface_to_s3, s3_iface_to_s2])
    swtich_net3 = slice.add_l2network(name=net_s1_s3_name, interfaces=[s3_iface_to_s1, s1_iface_to_s3])

    #Add host networks 
    host_net1 = slice.add_l2network(name=net_h1_name, interfaces=[s1_iface_local, h1_iface])
    host_net11 = slice.add_l2network(name=net_h11_name, interfaces=[s11_iface_local, h11_iface])

    host_net2 = slice.add_l2network(name=net_h2_name, interfaces=[s2_iface_local, h2_iface])
    host_net22 = slice.add_l2network(name=net_h22_name, interfaces=[s22_iface_local, h22_iface])
    
    host_net3 = slice.add_l2network(name=net_h3_name, interfaces=[s3_iface_local, h3_iface])
    
    #Submit Slice Request
    slice.submit() 
except Exception as e:
    print(f"Error: {e}")
    traceback.print_exc()
    


-----------  ------------------------------------
Slice Name   P4Lang_Tutorial
Slice ID     3d1ecbe2-1a64-419f-b939-8669f1175507
Slice State  StableOK
Lease End    2022-12-09 04:54:02 +0000
-----------  ------------------------------------

Retry: 11, Time: 285 sec

ID                                    Name    Site    Host                          Cores    RAM    Disk  Image              Management IP                        State    Error
------------------------------------  ------  ------  --------------------------  -------  -----  ------  -----------------  -----------------------------------  -------  -------
c6198056-3c4b-4846-ab11-7aa34879713d  s1      UTAH    utah-w2.fabric-testbed.net        2      8     100  default_ubuntu_20  2001:1948:417:7:f816:3eff:fed3:eba5  Active
ca2a5596-9a09-484b-b2ba-582199305b8b  s2      UTAH    utah-w2.fabric-testbed.net        2      8     100  default_ubuntu_20  2001:1948:417:7:f816:3eff:fe22:d8a7  Active
e0576c01-77d1-42fc-a701-d13ff4483758  

## Observe the Slice's Attributes

### Print the slice

In [5]:
try:
    slice = fablib.get_slice(name=slice_name)
    print(f"{slice}")
except Exception as e:
    print(f"Exception: {e}")

-----------  ------------------------------------
Slice Name   P4Lang_Tutorial
Slice ID     d1f8d6ca-1fb7-4e5f-8415-55c002073c12
Slice State  StableOK
Lease End    2022-11-14 08:52:41 +0000
-----------  ------------------------------------


### Print the Node List

In [6]:
try:
    slice = fablib.get_slice(name=slice_name)

    print(f"{slice.list_nodes()}")
except Exception as e:
    print(f"Exception: {e}")

ID                                    Name    Site    Host                          Cores    RAM    Disk  Image              Management IP                        State    Error
------------------------------------  ------  ------  --------------------------  -------  -----  ------  -----------------  -----------------------------------  -------  -------
fa451793-d220-43fe-844a-ec7fe817443e  s1      UTAH    utah-w5.fabric-testbed.net        2      8     100  default_ubuntu_20  2001:1948:417:7:f816:3eff:fed9:905c  Active
671ea5f2-d9bb-45e7-a9a1-b9e2a0cd5870  s2      UTAH    utah-w5.fabric-testbed.net        2      8     100  default_ubuntu_20  2001:1948:417:7:f816:3eff:fe9b:c307  Active
82c4119f-a1fb-4c07-aa20-2799c1b8b1a1  s3      UTAH    utah-w5.fabric-testbed.net        2      8     100  default_ubuntu_20  2001:1948:417:7:f816:3eff:feed:5934  Active
88705ab0-bdc7-4422-be5a-1484d6d37ab1  h1      UTAH    utah-w5.fabric-testbed.net        2      8      10  default_ubuntu_20  2001:1948:41

### Print the Node Details

In [7]:
try:
    slice = fablib.get_slice(name=slice_name)
    for node in slice.get_nodes():
        print(f"{node}")
except Exception as e:
    print(f"Exception: {e}")

-----------------  ------------------------------------------------------------------------------------------------------------------------------------------
ID                 fa451793-d220-43fe-844a-ec7fe817443e
Name               s1
Cores              2
RAM                8
Disk               100
Image              default_ubuntu_20
Image Type         qcow2
Host               utah-w5.fabric-testbed.net
Site               UTAH
Management IP      2001:1948:417:7:f816:3eff:fed9:905c
Reservation State  Active
Error Message
SSH Command        ssh -i /home/fabric/work/keys/jiabhi_sliver -J ajinda10_0051550143@bastion-1.fabric-testbed.net ubuntu@2001:1948:417:7:f816:3eff:fed9:905c
-----------------  ------------------------------------------------------------------------------------------------------------------------------------------
-----------------  -----------------------------------------------------------------------------------------------------------------------------------------

### Print the Node SSH Commands

In [8]:
try:
    slice = fablib.get_slice(name=slice_name)
    for node in slice.get_nodes():
        print(f"{node.get_name()}: {node.get_ssh_command()}")
except Exception as e:
    print(f"Exception: {e}")

s1: ssh -i /home/fabric/work/keys/jiabhi_sliver -J ajinda10_0051550143@bastion-1.fabric-testbed.net ubuntu@2001:1948:417:7:f816:3eff:fed9:905c
s2: ssh -i /home/fabric/work/keys/jiabhi_sliver -J ajinda10_0051550143@bastion-1.fabric-testbed.net ubuntu@2001:1948:417:7:f816:3eff:fe9b:c307
s3: ssh -i /home/fabric/work/keys/jiabhi_sliver -J ajinda10_0051550143@bastion-1.fabric-testbed.net ubuntu@2001:1948:417:7:f816:3eff:feed:5934
h1: ssh -i /home/fabric/work/keys/jiabhi_sliver -J ajinda10_0051550143@bastion-1.fabric-testbed.net ubuntu@2001:1948:417:7:f816:3eff:fe12:24b8
h11: ssh -i /home/fabric/work/keys/jiabhi_sliver -J ajinda10_0051550143@bastion-1.fabric-testbed.net ubuntu@2001:1948:417:7:f816:3eff:fe60:ddd4
h2: ssh -i /home/fabric/work/keys/jiabhi_sliver -J ajinda10_0051550143@bastion-1.fabric-testbed.net ubuntu@2001:1948:417:7:f816:3eff:fe2d:333f
h22: ssh -i /home/fabric/work/keys/jiabhi_sliver -J ajinda10_0051550143@bastion-1.fabric-testbed.net ubuntu@2001:1948:417:7:f816:3eff:fe88:ab

### Print the Interfaces

In [9]:
try:
    slice = fablib.get_slice(name=slice_name)
    
    print(f"{slice.list_interfaces()}")
except Exception as e:
    print(f"Exception: {e}")

Name                  Node    Network      Bandwidth  VLAN    MAC                Physical OS Interface    OS Interface
--------------------  ------  ---------  -----------  ------  -----------------  -----------------------  --------------
s1-s1_switch_nic1-p1  s1      net_s1_s2            0          0A:CE:D5:B5:F0:09  ens10                    ens10
s1-s1_local_nic-p1    s1      net_h1               0          0A:5C:7E:54:66:E7  ens9                     ens9
s1-s1_switch_nic2-p1  s1      net_s1_s3            0          06:E6:02:2D:02:57  ens8                     ens8
s1-s11_local_nic-p1   s1      net_h11              0          06:1F:9F:D4:E2:30  ens7                     ens7
s2-s2_switch_nic2-p1  s2      net_s2_s3            0          02:29:C7:8C:E1:88  ens8                     ens8
s2-s22_local_nic-p1   s2      net_h22              0          02:AA:1F:20:41:A6  ens9                     ens9
s2-s2_switch_nic1-p1  s2      net_s1_s2            0          02:13:80:A3:DC:9D  ens7        

## Configure Nodes


In [10]:
config_threads = {}

In [11]:
host_config_script = "sudo apt-get update -qq && sudo apt-get install -qq -y python3-scapy && git clone https://github.com/p4lang/tutorials.git/" 

try:
    
    
    h1 = slice.get_node(name=h1_name)        
    if type(ip_address(h1.get_management_ip())) is IPv6Address:
        h1.execute("sudo sed -i '/nameserver/d' /etc/resolv.conf && sudo sh -c 'echo nameserver 2a00:1098:2c::1 >> /etc/resolv.conf' && sudo sh -c 'echo nameserver 2a01:4f8:c2c:123f::1 >> /etc/resolv.conf' && sudo sh -c 'echo nameserver 2a00:1098:2b::1 >> /etc/resolv.conf'")
       
    h1_os_iface = h1.get_interface(network_name=net_h1_name)
    h1_os_iface.ip_addr_add(addr=h1_addr, subnet=h1_subnet)
    h1_config_thread = h1.execute(host_config_script)
    # config_threads[h1] = h1_config_thread
    
    h11 = slice.get_node(name=h11_name)        
    if type(ip_address(h11.get_management_ip())) is IPv6Address:
        h11.execute("sudo sed -i '/nameserver/d' /etc/resolv.conf && sudo sh -c 'echo nameserver 2a00:1098:2c::1 >> /etc/resolv.conf' && sudo sh -c 'echo nameserver 2a01:4f8:c2c:123f::1 >> /etc/resolv.conf' && sudo sh -c 'echo nameserver 2a00:1098:2b::1 >> /etc/resolv.conf'")

    h11_os_iface = h11.get_interface(network_name=net_h11_name)
    h11_os_iface.ip_addr_add(addr=h11_addr, subnet=h11_subnet)
    h11_config_thread = h11.execute(host_config_script)
    # config_threads[h11] = h11_config_thread

    
    h2 = slice.get_node(name=h2_name)
    if type(ip_address(h2.get_management_ip())) is IPv6Address:
        h2.execute("sudo sed -i '/nameserver/d' /etc/resolv.conf && sudo sh -c 'echo nameserver 2a00:1098:2c::1 >> /etc/resolv.conf' && sudo sh -c 'echo nameserver 2a01:4f8:c2c:123f::1 >> /etc/resolv.conf' && sudo sh -c 'echo nameserver 2a00:1098:2b::1 >> /etc/resolv.conf'")
 
    h2_os_iface = h2.get_interface(network_name=net_h2_name)
    h2_os_iface.ip_addr_add(addr=h2_addr, subnet=h2_subnet)
    h2_config_thread = h2.execute(host_config_script)
    # config_threads[h2] = h2_config_thread
    
    h22 = slice.get_node(name=h22_name)
    if type(ip_address(h22.get_management_ip())) is IPv6Address:
        h22.execute("sudo sed -i '/nameserver/d' /etc/resolv.conf && sudo sh -c 'echo nameserver 2a00:1098:2c::1 >> /etc/resolv.conf' && sudo sh -c 'echo nameserver 2a01:4f8:c2c:123f::1 >> /etc/resolv.conf' && sudo sh -c 'echo nameserver 2a00:1098:2b::1 >> /etc/resolv.conf'")
 
    h22_os_iface = h22.get_interface(network_name=net_h22_name)
    h22_os_iface.ip_addr_add(addr=h22_addr, subnet=h22_subnet)
    h22_config_thread = h22.execute(host_config_script)
    # config_threads[h22] = h22_config_thread

    h3 = slice.get_node(name=h3_name)
    if type(ip_address(h3.get_management_ip())) is IPv6Address:
        h3.execute("sudo sed -i '/nameserver/d' /etc/resolv.conf && sudo sh -c 'echo nameserver 2a00:1098:2c::1 >> /etc/resolv.conf' && sudo sh -c 'echo nameserver 2a01:4f8:c2c:123f::1 >> /etc/resolv.conf' && sudo sh -c 'echo nameserver 2a00:1098:2b::1 >> /etc/resolv.conf'")
 
    h3_os_iface = h3.get_interface(network_name=net_h3_name)
    h3_os_iface.ip_addr_add(addr=h3_addr, subnet=h3_subnet)
    h3_config_thread = h3.execute(host_config_script)
    # config_threads[h3] = h3_config_thread

except Exception as e:
    print(f"Error: {e}")
    traceback.print_exc()

## Configure Switches

Use ssh to configure the ifaces on the switches. This step requires testing the interfaces to figure out which interface is connected to which network.


#### Setup P4 Docker



Below are commands to let sudo work with the global proxy.

In [12]:
try:
    s1 = slice.get_node(name=s1_name)
    s1_h1_os_iface = s1.get_interface(network_name=net_h1_name)
    s1_h11_os_iface = s1.get_interface(network_name=net_h11_name)
    s1_s2_os_iface = s1.get_interface(network_name=net_s1_s2_name)
    s1_s3_os_iface = s1.get_interface(network_name=net_s1_s3_name)
    
    print(s1_s2_os_iface)
    
    if type(ip_address(s1.get_management_ip())) is IPv4Address:
        management_ip_type = "IPv4"
    else:
        management_ip_type = "IPv6"
    
    file_attributes = s1.upload_file('./scripts/router_setup_p4_bmv2_container.sh','router_setup_p4_bmv2_container.sh')
    command=f"chmod +x router_setup_p4_bmv2_container.sh && sudo sh -c './router_setup_p4_bmv2_container.sh  {management_ip_type} {s1_h1_os_iface.get_os_interface()} {s1_h11_os_iface.get_os_interface()} {s1_s2_os_iface.get_os_interface()} {s1_s3_os_iface.get_os_interface()}  > /tmp/script.log 2>&1'"
    print(command)
    s1_config_thread = s1.execute(command)
    # config_threads[s1] = s1_config_thread


except Exception as e:
    print(f"Error: {e}")
    traceback.print_exc()

---------------------  --------------------
Name                   s1-s1_switch_nic1-p1
Network                net_s1_s2
Bandwidth              0
VLAN
MAC                    0A:CE:D5:B5:F0:09
Physical OS Interface  ens10
OS Interface           ens10
---------------------  --------------------
chmod +x router_setup_p4_bmv2_container.sh && sudo sh -c './router_setup_p4_bmv2_container.sh  IPv6 ens9 ens7 ens10 ens8  > /tmp/script.log 2>&1'


In [13]:
try:
    s2 = slice.get_node(name=s2_name)
    # print(s2.get_interfaces())
    s2_h2_os_iface = s2.get_interface(network_name=net_h2_name)
    s2_h22_os_iface = s2.get_interface(network_name=net_h22_name)
    s2_s1_os_iface = s2.get_interface(network_name=net_s1_s2_name)
    print(s2_s1_os_iface.get_physical_os_interface())
    
    s2_s3_os_iface = s2.get_interface(network_name=net_s2_s3_name)

    if type(ip_address(s2.get_management_ip())) is IPv4Address:
        management_ip_type = "IPv4"
    else:
        management_ip_type = "IPv6"
    
    file_attributes = s2.upload_file('scripts/router_setup_p4_bmv2_container.sh','router_setup_p4_bmv2_container.sh')
    command= f"chmod +x router_setup_p4_bmv2_container.sh && sudo sh -c './router_setup_p4_bmv2_container.sh  {management_ip_type} {s2_h2_os_iface.get_os_interface()} {s2_h22_os_iface.get_os_interface()} {s2_s1_os_iface.get_os_interface()} {s2_s3_os_iface.get_os_interface()}  > /tmp/script.log 2>&1'"
    print(command)
    s2_config_thread = s2.execute(command)
    # config_threads[s2] = s2_config_thread

except Exception as e:
    print(f"Error: {e}")
    traceback.print_exc() 

None
chmod +x router_setup_p4_bmv2_container.sh && sudo sh -c './router_setup_p4_bmv2_container.sh  IPv6 ens10 ens9 ens7 ens8  > /tmp/script.log 2>&1'


In [14]:
try:
    s3 = slice.get_node(name=s3_name)
    s3_h3_os_iface = s3.get_interface(network_name=net_h3_name)
    s3_s1_os_iface = s3.get_interface(network_name=net_s1_s3_name)
    s3_s2_os_iface = s3.get_interface(network_name=net_s2_s3_name)

    if type(ip_address(s3.get_management_ip())) is IPv4Address:
        management_ip_type = "IPv4"
    else:
        management_ip_type = "IPv6"
     
    file_attributes = s3.upload_file('scripts/router_setup_p4_bmv2_container.sh','router_setup_p4_bmv2_container.sh')
    command = f"chmod +x router_setup_p4_bmv2_container.sh && sudo sh -c './router_setup_p4_bmv2_container.sh  {management_ip_type} {s3_h3_os_iface.get_os_interface()} {s3_s1_os_iface.get_os_interface()} {s3_s2_os_iface.get_os_interface()} > /tmp/script.log 2>&1'"
    print(command)
    s3_config_thread = s3.execute(command)
    # config_threads[s3] = s3_config_thread


except Exception as e:
    print(f"Error: {e}")
    traceback.print_exc() 

chmod +x router_setup_p4_bmv2_container.sh && sudo sh -c './router_setup_p4_bmv2_container.sh  IPv6 ens8 ens9 ens7 > /tmp/script.log 2>&1'


In [15]:
try:
    for node, thread in config_threads.items():
        stdout, stderr = thread.result()
        print(f"Config thread node {node.get_name()} complete")
        print(f"stdout: {stdout}")
        print(f"stderr: {stderr}")
except Exception as e:
    print(f"Error: {e}")
    traceback.print_exc() 


### Confgure P4 Switch Tables

Edit sX_commands.txt to change the values

In [17]:
for switch_name in [s1_name, s2_name, s3_name]:
    switch_node = slice.get_node(name=switch_name)
    management_ip_switch = str(switch_node.get_management_ip())
    print("Swtitch Name        : {}".format(switch_node.get_name()))
    print("Management IP    : {}".format(management_ip_switch))
    
    #Configure P4 Tables
    cmd_file=f'{switch_name}_commands.txt'
    print(cmd_file)
    file_attributes = switch_node.upload_file(f'scripts/{cmd_file}',cmd_file)
    print("file_attributes: {}".format(file_attributes))
    print(f"sudo sh -c 'cat {cmd_file} | docker exec -i fabric_p4 simple_switch_CLI'")
    stdout = switch_node.execute(f"sudo sh -c 'cat {cmd_file} | docker exec -i fabric_p4 simple_switch_CLI'")
    print("stdout: {}".format(stdout))

Swtitch Name        : s1
Management IP    : 2001:1948:417:7:f816:3eff:fed9:905c
s1_commands.txt
file_attributes: -rw-rw-r--   1 1000     1000          185 13 Nov 09:35 ?
sudo sh -c 'cat s1_commands.txt | docker exec -i fabric_p4 simple_switch_CLI'
stdout: ('Obtaining JSON from switch...\nDone\nControl utility for runtime P4 table manipulation\nRuntimeCmd: Setting default action of myTunnel_exact\naction:              drop\nruntime data:        \nRuntimeCmd: Adding entry to exact match table myTunnel_exact\nmatch key:           EXACT-00:01\naction:              myTunnel_forward\nruntime data:        00:01\nEntry has been added with handle 0\nRuntimeCmd: Adding entry to exact match table myTunnel_exact\nmatch key:           EXACT-00:02\naction:              myTunnel_forward\nruntime data:        00:02\nEntry has been added with handle 1\nRuntimeCmd: Adding entry to exact match table myTunnel_exact\nmatch key:           EXACT-00:03\naction:              myTunnel_forward\nruntime data:    

## The switches are now configured and running. Now we are going to send packets over the switches.

We are going to use `send.py` and `receive.py`. We are going to re-upload them to the servers and use them. Make sure to modify the interface names in the script accordingly.

In [18]:
h1.upload_file('scripts/send.py', 'tutorials/exercises/basic_tunnel/send_modified.py')
h2.upload_file('scripts/receive.py', 'tutorials/exercises/basic_tunnel/receive_modified.py')

#h1.execute('sudo mv send.py tutorials/exercises/basic_tunnel/send_modified.py')
#h2.execute('sudo mv receive.py tutorials/exercises/basic_tunnel/receive_modified.py')

<SFTPAttributes: [ size=670 uid=1000 gid=1000 mode=0o100664 atime=1668332175 mtime=1668332175 ]>

## Below, we send a few packets.

In [19]:
h1.execute('tmux new -d \'timeout 30 watch -n 5 sudo python3 tutorials/exercises/basic_tunnel/send_modified.py --dst_id 2 10.10.2.2 "message100"\'')

('', '')

## And below we receive them.

In [20]:
print(h2.execute('sudo timeout 30 sudo python3 tutorials/exercises/basic_tunnel/receive_modified.py')[0])

## Delete Slice

In [ ]:
try:
    slice = fablib.get_slice(name=slice_name)
    slice.delete()
except Exception as e:
    print(f"Fail: {e}")
    traceback.print_exc()